In [1]:
import pandas as pd
import numpy as np

import sonny_acquire
import prepare

In [2]:
df = sonny_acquire.scrape_github_data()

In [3]:
df = pd.DataFrame(df)

In [4]:
df

,repo,language,readme_contents
0,lucaseustaquio/ams-2013-2014-solar-energy,R,ams-2013-2014-solar-energy\n==================...
1,renewables-ninja/gsee,Python,[![Master branch build status](https://img.shi...
2,ColasGael/Machine-Learning-for-Solar-Energy-Pr...,Python,# Machine-Learning-for-Solar-Energy-Prediction...
3,owenzhang/kaggle_AMS_2013_14_solar,None,"!!! Read this file in ""code"" or ""Raw"" mode\n\n..."
4,meltaxa/solariot,Python,# Solariot\n\nLeverage your IoT enabled Solar ...
...,...,...,...
95,GigaJunky/TeslaSolarChart,JavaScript,\n# Tesla Solar Energy Monitor Charts and Prox...
96,nicolasfguillaume/3D-Solar-Tracker-IoT-Arduino,C,# 3D-printed Single-axis solar tracker with En...
97,iobroker-community-adapters/ioBroker.vedirect,JavaScript,![Logo](admin/vedirect.png)\r\n# ioBroker.vedi...
98,deepakatl/blockchain-energy-live,JavaScript,# blockchain-energy-live\nSolar energy blockch...


In [5]:
df = prepare.prep_repo_data(df)

In [6]:
df

,repo,language,readme_contents,clean_stemmed,clean_lemmatized
0,lucaseustaquio/ams-2013-2014-solar-energy,R,ams-2013-2014-solar-energy\n==================...,ams20132014solarenergi read modeldocumentation...,ams20132014solarenergy read modeldocumentation...
1,renewables-ninja/gsee,Python,[![Master branch build status](https://img.shi...,master branch build statushttpsimgshieldsioazu...,master branch build statushttpsimgshieldsioazu...
2,ColasGael/Machine-Learning-for-Solar-Energy-Pr...,Python,# Machine-Learning-for-Solar-Energy-Prediction...,machinelearningforsolarenergypredict adel kuzm...,machinelearningforsolarenergyprediction adele ...
3,owenzhang/kaggle_AMS_2013_14_solar,None,"!!! Read this file in ""code"" or ""Raw"" mode\n\n...",read thi file code raw mode run 1 set folder f...,read file code raw mode run 1 set folder follo...
4,meltaxa/solariot,Python,# Solariot\n\nLeverage your IoT enabled Solar ...,solariot leverag iot enabl solar pv invert str...,solariot leverage iot enabled solar pv inverte...
...,...,...,...,...,...
95,GigaJunky/TeslaSolarChart,JavaScript,\n# Tesla Solar Energy Monitor Charts and Prox...,tesla solar energi monitor chart proxi web ser...,tesla solar energy monitor chart proxy web ser...
96,nicolasfguillaume/3D-Solar-Tracker-IoT-Arduino,C,# 3D-printed Single-axis solar tracker with En...,3dprint singleaxi solar tracker energi storag ...,3dprinted singleaxis solar tracker energy stor...
97,iobroker-community-adapters/ioBroker.vedirect,JavaScript,![Logo](admin/vedirect.png)\r\n# ioBroker.vedi...,logoadminvedirectpng iobrokervedirect npm vers...,logoadminvedirectpng iobrokervedirect npm vers...
98,deepakatl/blockchain-energy-live,JavaScript,# blockchain-energy-live\nSolar energy blockch...,blockchainenergyl solar energi blockchain,blockchainenergylive solar energy blockchain
